In [1]:
import pickle

import pandas as pd

import numpy as np

from datetime import datetime

import os

In [2]:
with open ("sensor_stats.pkl" , "rb") as f:
    
    data = pickle.load(f)
    
print(data['sensor_17'])

{'mean': 421.1275168969532, 'min': 0.0, 'max': 576.4103487694446, 'std': 123.02874288164513, 'anomaly_count': 0.058823529411764705, 'anomaly_threshold': 544.1562597785983}


In [3]:
with open("model.pkl" , "rb") as model_f:
    
    prediction_model = pickle.load(model_f)

In [4]:
with open ("X_train_columns.pkl" , 'rb') as x_train_columns_file:
    
    columns_info = pickle.load(x_train_columns_file)
    
x_train_columns_file.close
    
columns_info

Index(['sensor_00_anomalycount', 'sensor_01_anomalycount',
       'sensor_02_anomalycount', 'sensor_03_anomalycount',
       'sensor_04_anomalycount', 'sensor_05_anomalycount',
       'sensor_06_anomalycount', 'sensor_07_anomalycount',
       'sensor_08_anomalycount', 'sensor_09_anomalycount',
       ...
       'sensor_49_max', 'sensor_49_std', 'sensor_50_mean', 'sensor_50_min',
       'sensor_50_max', 'sensor_50_std', 'sensor_51_mean', 'sensor_51_min',
       'sensor_51_max', 'sensor_51_std'],
      dtype='object', length=260)

In [5]:
with open ("std_values.pkl" , 'rb') as std_values_file:
    
    std_values = pickle.load(std_values_file)
    
std_values_file.close

<function BufferedReader.close>

In [6]:
std_values

{'sensor_00': 0.3172427797757728,
 'sensor_01': 3.0014634521497094,
 'sensor_02': 3.5553888653992742,
 'sensor_03': 2.154136369116654,
 'sensor_04': 135.18676924536982,
 'sensor_05': 13.361702195707812,
 'sensor_06': 1.7587710884753263,
 'sensor_07': 1.5984173928779932,
 'sensor_08': 1.2650128252945434,
 'sensor_09': 1.5410394327201478,
 'sensor_10': 11.032223154792806,
 'sensor_11': 12.065440445768967,
 'sensor_12': 8.84348956175112,
 'sensor_13': 6.08254201862875,
 'sensor_14': 109.80114425434138,
 'sensor_15': 0.009693863576651458,
 'sensor_16': 121.09030902629034,
 'sensor_17': 123.02874288164513,
 'sensor_18': 0.7105211657625926,
 'sensor_19': 191.51765497725233,
 'sensor_20': 97.65758739256607,
 'sensor_21': 215.62306598338103,
 'sensor_22': 148.31220809529233,
 'sensor_23': 279.8892642491386,
 'sensor_24': 172.51471564350823,
 'sensor_25': 212.2382417072381,
 'sensor_26': 234.53514655656957,
 'sensor_27': 122.15319701043683,
 'sensor_28': 286.3818155496953,
 'sensor_29': 210.021

In [7]:
def input_preprocessing(sensor_values):
    
    with open ("sensor_stats.pkl" , "rb") as f:
        
        data = pickle.load(f)
    
    processed_data = {}
    
    anomalies_data = {}
    
    for i in range(52):
        
        sensor_name = f"sensor_{i:02d}"
        
        anomaly_val = data[sensor_name]['anomaly_count']
        
        processed_data[f"{sensor_name}_anomalycount"] = anomaly_val
    
    
    
    for j in range(52):
        
        sensor_name = f"sensor_{j:02d}"
        
        mean_val = data[sensor_name]['mean']
        
        min_val = data[sensor_name]["min"]
        
        max_val = data[sensor_name]["max"]
        
        std_val = data[sensor_name]['std']
        
        
        anomaly_threshold_limit = data[sensor_name]['anomaly_threshold']
        
            
            
        
        processed_data[f"{sensor_name}_mean"] = sensor_values[j]
        
        processed_data[f"{sensor_name}_min"] = min_val
        
        processed_data[f"{sensor_name}_max"] = max_val
        
        processed_data[f"{sensor_name}_std"] = std_val
        
                
        if std_val == 0:
            
            z_score = 0  
            
        else:
             
            z_score = (sensor_values[j] - mean_val) / std_val
            
            
        is_anomaly = int(abs(z_score) > anomaly_threshold_limit)
        
        if is_anomaly:
            
            anomalies_data[sensor_name] = sensor_values[j] 
            
        print(f"Sensor {sensor_name}: Z-score = {z_score}, Threshold = {anomaly_threshold_limit}")

        
        
    
    return processed_data , anomalies_data
    

In [8]:
sensor_data = np.array([2.37, 0.18, 2.52, 0.32, 47.59, 34.19, 53.67, 3.00, 50.87, 33.92, 54.49, 3.56, 
    43.75, 34.19, 46.35, 2.15, 590.67, 3.21, 640.36, 135.19, 73.40, 0.00, 99.99, 13.36, 
    13.50, 1.79, 15.11, 1.76, 15.84, 2.74, 17.40, 1.60, 15.20, 5.91, 16.96, 1.27, 
    14.80, 2.54, 16.34, 1.54, 41.47, 0.25, 50.71, 11.03, 41.92, 0.04, 54.31, 12.07, 
    29.14, 0.00, 37.38, 8.84, 7.08, 0.00, 19.26, 6.08, 376.86, 37.36, 429.85, 109.80, 
    1.35, 1.28, 1.45, 0.01, 416.47, 24.15, 467.92, 121.09, 421.13, 0.00, 576.41, 123.03, 
    2.30, 0.00, 4.02, 0.71, 590.83, 0.00, 676.50, 191.52, 360.81, 60.62, 405.88, 97.66, 
    796.23, 127.82, 894.43, 215.62, 459.79, 0.00, 542.29, 148.31, 922.61, 53.10, 1108.19, 279.89, 
    556.24, 0.05, 670.81, 172.51, 649.14, 0.69, 780.22, 212.24, 786.41, 82.39, 983.15, 234.54, 
    501.51, 124.07, 987.15, 122.15, 851.69, 54.85, 1206.29, 286.38, 576.20, 2.55, 1022.41, 210.02, 
    614.60, 80.90, 912.94, 173.92, 863.32, 104.22, 1580.89, 251.29, 804.28, 90.04, 1466.67, 239.32, 
    486.41, 47.90, 730.02, 137.89, 234.97, 61.82, 359.86, 84.67, 427.13, 55.79, 602.88, 134.15, 
    593.03, 69.80, 909.58, 277.53, 60.79, 1.77, 109.55, 32.05, 49.66, 26.04, 67.49, 5.69, 
    36.61, 21.38, 97.91, 6.95, 68.84, 26.00, 94.20, 11.99, 35.37, 22.49, 58.87, 3.81, 
    35.45, 24.84, 68.22, 4.11, 43.88, 27.07, 69.78, 4.40, 42.66, 28.18, 68.36, 6.05, 
    43.09, 28.71, 66.21, 6.31, 48.02, 28.75, 65.18, 7.03, 44.34, 29.62, 58.41, 4.59, 
    150.89, 28.60, 313.54, 71.30, 57.12, 28.90, 119.21, 13.67, 183.05, 29.69, 259.81, 44.67, 
    202.70, 30.20, 1000.00, 85.87])


processed_data , anomaly_data = input_preprocessing(sensor_data)

Sensor sensor_00: Z-score = -0.007000366129422813, Threshold = 2.689463595386119
Sensor sensor_01: Z-score = -15.796164589817382, Threshold = 50.593074152627985
Sensor sensor_02: Z-score = -13.598341553933565, Threshold = 54.422781014150935
Sensor sensor_03: Z-score = -20.162363744452136, Threshold = 45.906617398400044
Sensor sensor_04: Z-score = -4.017286151074552, Threshold = 725.8607051433053
Sensor sensor_05: Z-score = -2.9342379674746564, Threshold = 86.75811608844315
Sensor sensor_06: Z-score = 22.838937266697197, Threshold = 15.26030853230661
Sensor sensor_07: Z-score = -8.034917491548894, Threshold = 17.44156926170936
Sensor sensor_08: Z-score = 28.196772411614315, Threshold = 16.46573409269108
Sensor sensor_09: Z-score = 12.407722986923558, Threshold = 16.340249039602732
Sensor sensor_10: Z-score = 1.1801484434880598, Threshold = 52.50256217045114
Sensor sensor_11: Z-score = -3.179189291288315, Threshold = 53.98375950563458
Sensor sensor_12: Z-score = 1.652404876466246, Thresh

In [9]:
len(processed_data)

260

In [10]:
X_test = pd.DataFrame([processed_data])

output_mapping = {0 : "Broken / Inefficient",
                 1 : "Working Fine"}

In [11]:
X_test

,sensor_00_anomalycount,sensor_01_anomalycount,sensor_02_anomalycount,sensor_03_anomalycount,sensor_04_anomalycount,sensor_05_anomalycount,sensor_06_anomalycount,sensor_07_anomalycount,sensor_08_anomalycount,sensor_09_anomalycount,...,sensor_49_max,sensor_49_std,sensor_50_mean,sensor_50_min,sensor_50_max,sensor_50_std,sensor_51_mean,sensor_51_min,sensor_51_max,sensor_51_std
0,0.019608,0.026144,0.039216,0.03268,0.052288,0.026144,0.019608,0.026144,0.019608,0.026144,...,119.208988,13.665849,37.38,29.691117,259.809227,44.665949,8.84,30.204929,1000.0,85.869828


In [12]:
processed_data

{'sensor_00_anomalycount': 0.0196078431372549,
 'sensor_01_anomalycount': 0.026143790849673203,
 'sensor_02_anomalycount': 0.0392156862745098,
 'sensor_03_anomalycount': 0.032679738562091505,
 'sensor_04_anomalycount': 0.05228758169934641,
 'sensor_05_anomalycount': 0.026143790849673203,
 'sensor_06_anomalycount': 0.0196078431372549,
 'sensor_07_anomalycount': 0.026143790849673203,
 'sensor_08_anomalycount': 0.0196078431372549,
 'sensor_09_anomalycount': 0.026143790849673203,
 'sensor_10_anomalycount': 0.058823529411764705,
 'sensor_11_anomalycount': 0.05228758169934641,
 'sensor_12_anomalycount': 0.05228758169934641,
 'sensor_13_anomalycount': 0.0,
 'sensor_14_anomalycount': 0.06535947712418301,
 'sensor_15_anomalycount': 0.013071895424836602,
 'sensor_16_anomalycount': 0.06535947712418301,
 'sensor_17_anomalycount': 0.058823529411764705,
 'sensor_18_anomalycount': 0.058823529411764705,
 'sensor_19_anomalycount': 0.06535947712418301,
 'sensor_20_anomalycount': 0.0718954248366013,
 'se

In [13]:
prediction_result = prediction_model.predict(X_test)

In [14]:
final_result = output_mapping[int(prediction_result[0])]

print(f"Status Of The Machine : {final_result}")

Status Of The Machine : Working Fine


In [15]:
anomaly_data

{'sensor_06': 53.67,
 'sensor_08': 50.87,
 'sensor_15': 2.15,
 'sensor_18': 640.36}

In [16]:
with open("correlations_data.pkl" , 'rb') as file:
        
    correlation_data = pickle.load(file)
        
file.close

<function BufferedReader.close>

In [17]:
correlation_data

,Sensor_A,Sensor_B,Correlation
0,Motor Frequency B,Motor Frequency C,0.822339
1,Motor Frequency B,Motor Speed,0.820339
2,Motor Speed,Motor Phase Current A,0.907521
3,Motor Speed,Motor Phase Current B,0.862619
4,Motor Speed,Motor Phase Current C,0.803165
...,...,...,...
121,Pump Stage 2 Impeller Speed,Pump Stage 2 Impeller Speed,0.865460
122,Pump Stage 2 Impeller Speed,Pump Stage 2 Impeller Speed,0.846420
123,Pump Stage 2 Impeller Speed,Pump Stage 2 Impeller Speed,0.854616
124,Pump Inlet Flow,Pump Discharge Flow,0.810734


In [18]:
def excel_report_generator(anomalies ,machine_status , plot_folder="C:/Users/Abhi/ControlIT_JaajiTech/Sensor_Variations"):
    
    with open("correlations_data.pkl" , 'rb') as file:
        
        correlation_data = pickle.load(file)
                
    with open("sensor_mappings.pkl" , 'rb') as f:
        
        sensor_names_data = pickle.load(f)
            
    report_data = []
    
    for sensor , value in anomalies.items():
        
        actual_sensor_name = sensor_names_data.get(sensor , sensor)
        
        correlated_sensors = correlation_data[(correlation_data["Sensor_A"] == actual_sensor_name) | (correlation_data["Sensor_B"] == actual_sensor_name)].sort_values(by="Correlation" , ascending = False)
        
        unique_correlated_sensors = set()
        
        for _, row in correlated_sensors.iterrows():
            
            correlated_sensor = row["Sensor_B"] if row["Sensor_A"] == actual_sensor_name else row["Sensor_A"]
            
            unique_correlated_sensors.add(correlated_sensor) 
            
            if len(unique_correlated_sensors) >= 3:  
                
                break      
                
        plot_filename = f"plot_{sensor}.png"
        
        plot_path = os.path.join(plot_folder , plot_filename)
        
        plot_link = "NA"
        
        if os.path.exists(plot_path):
            
            plot_link = f'=HYPERLINK("{plot_path}" ,"View_Plot" )'
                
        correlated_sensors_str = " , ".join(unique_correlated_sensors)
        
        machine_current_status = f"Working Status : {machine_status}"
        
        report_data.append([sensor , actual_sensor_name , value , correlated_sensors_str , plot_link , machine_current_status])
        

        
        df = pd.DataFrame(report_data , columns = ["Sensor_ID" , "Sensor_Name" , "Anomalous_Sensor_Value" , "Correlated_Sensors", "Plot_Link","Machine_Current_Status"])
        
        df["Detection_Time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        

    
    file_name = f"Anomaly_Report_{datetime.now().strftime('%Y-%m-%d %H %M %S')}.xlsx"

    df.to_excel(file_name)
    
    print(f"Report Saved Successfully at {file_name}")
    
    return file_name
    
                                                          
                      
        
    
    
    

In [19]:
if anomaly_data:
    
    excel_report_generator(anomaly_data,final_result)

Report Saved Successfully at Anomaly_Report_2025-02-28 09 58 33.xlsx
